<a href="https://colab.research.google.com/github/Maria-lin/How-to-make-notebook-in-dataiku/blob/main/notebook_dataiku__explique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Dataiku — Analyse exploratoire & Détection d’anomalies (DAB / GAB)

Ce notebook est conçu pour **Dataiku DSS** et pour ton cas : **transactions de retraits** agrégées par automate.  
Il fait :

1. **Chargement Dataiku**  
2. **Audit qualité** (types, valeurs manquantes, incohérences)  
3. **EDA profonde** (stats + plots)  
4. **Features non évidentes** (ratios/cohérences)  
5. **Détection d’anomalies non supervisée** (Isolation Forest)  
6. **Sortie actionnable** : *Top anomalies* + raisons simples + export vers Dataiku (optionnel)

✅ **Important :** Tu n’as **pas** de colonne `anomalie`, donc on **n’utilise pas** de matrice de confusion.  
À la place, on fait une **validation non supervisée** : cohérence des distributions, stabilité, interprétation.

---

## Colonnes attendues (d’après ce que tu as donné)

- `num_automate` (int64)  
- `lib_site_implementation` (object)  
- `code_banque` (int64)  
- `type_carte` (object)  
- `montant_total` (float)  
- `nb_total_de_retraits` (int)  
- `type_gab_e_i` (int : 2 valeurs possibles, ex : E/I)  
- `code_postale_emplacement` (int64)  
- `dab_hos_site` (object : `B`=bureau / `H`=hors site)  
- `typ_gab` (object : retraits uniquement)

Si une colonne manque, certaines cellules afficheront un message et passeront.


In [ ]:
# === 0) Imports & options d'affichage ===
# Cette cellule importe les librairies nécessaires et configure l'affichage.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import IsolationForest

pd.set_option("display.max_columns", 200)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

# Pour avoir des résultats reproductibles
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


## 1) Chargement des données (Dataiku)

**Ce que fait le code :**
- Récupère un dataset Dataiku par son nom
- Le convertit en DataFrame pandas `df`

**Sortie attendue :**
- `Shape` (nb lignes/colonnes)
- aperçu des premières lignes (`df.head()`)

⚠️ **Action pour toi :** remplace `DATASET_NAME` par le nom exact de ton dataset dans Dataiku.


In [ ]:
import dataiku

# 🔧 À MODIFIER : nom exact du dataset Dataiku
DATASET_NAME = "TON_DATASET_DAB"

dataset = dataiku.Dataset(DATASET_NAME)
df = dataset.get_dataframe()

print("Shape (lignes, colonnes) =", df.shape)
df.head()


## 2) Audit qualité (senior-style)

On vérifie systématiquement :
- **Types** et colonnes présentes
- **Valeurs manquantes**
- **Doublons**
- **Incohérences** (ex : montant_total > 0 alors que nb_total_de_retraits == 0)

**Sorties attendues :**
- `df.info()` : types + non-null counts
- tableau des % de valeurs manquantes
- compte des doublons
- compte d’incohérences


In [ ]:
# 2.1 Types & complétude
df.info()


In [ ]:
# 2.2 Valeurs manquantes (% par colonne)
missing_pct = (df.isna().mean().sort_values(ascending=False) * 100).round(2)
missing_pct[missing_pct > 0]


In [ ]:
# 2.3 Doublons
dup_rows = int(df.duplicated().sum())
dup_automate = int(df.duplicated(subset=["num_automate"]).sum()) if "num_automate" in df.columns else None
{"doublons_lignes": dup_rows, "doublons_num_automate": dup_automate}


In [ ]:
# 2.4 Contrôles de cohérence (adaptables)
checks = {}

if "montant_total" in df.columns:
    checks["montant_total_negatif"] = int((df["montant_total"] < 0).sum())
    checks["montant_total_zero"] = int((df["montant_total"] == 0).sum())

if "nb_total_de_retraits" in df.columns:
    checks["nb_retraits_negatif"] = int((df["nb_total_de_retraits"] < 0).sum())
    checks["nb_retraits_zero"] = int((df["nb_total_de_retraits"] == 0).sum())

# incohérence : montant positif mais 0 retrait
if "montant_total" in df.columns and "nb_total_de_retraits" in df.columns:
    checks["incoherence_montant_pos_nb0"] = int(((df["montant_total"] > 0) & (df["nb_total_de_retraits"] == 0)).sum())

checks


## 3) Comprendre les variables catégorielles

**Ce que fait le code :**
- Liste les modalités (top) et leurs fréquences pour :
  - `lib_site_implementation`
  - `type_carte`
  - `dab_hos_site`
  - `typ_gab`

**Pourquoi c’est utile :**
- Détecter des valeurs inattendues (ex : typo, nouvelle modalité)
- Vérifier si une variable est constante (ex : `typ_gab` toujours "RETRAIT")

**Sortie attendue :**
- `n_unique` + top 15 des valeurs


In [ ]:
cat_cols = [c for c in ["lib_site_implementation","type_carte","dab_hos_site","typ_gab"] if c in df.columns]

for c in cat_cols:
    print("\n===", c, "===")
    print("n_unique:", df[c].nunique(dropna=False))
    display(df[c].value_counts(dropna=False).head(15))


## 4) Statistiques descriptives des variables numériques

**Ce que fait le code :**
- Résume les variables numériques avec percentiles (1%, 5%, 50%, 95%, 99%).

**Pourquoi :**
- Les percentiles aident à voir les extrêmes sans inventer des seuils arbitraires.

**Sortie attendue :**
- Tableau `describe()` transposé


In [ ]:
num_cols = [c for c in [
    "montant_total",
    "nb_total_de_retraits",
    "type_gab_e_i",
    "code_postale_emplacement",
    "code_banque"
] if c in df.columns]

df[num_cols].describe(percentiles=[.01,.05,.25,.5,.75,.95,.99]).T


## 5) Visualisations EDA (plots “qui parlent”)

On veut voir :
- **la forme** des distributions (asymétrie, queue à droite)
- la relation **montant_total vs nb_total_de_retraits**
- une comparaison **B vs H** (bureau vs hors site) si disponible

**Sorties attendues :**
- histogramme nb_total_de_retraits
- histogramme log(1+montant_total)
- scatter plot (nuage de points)


In [ ]:
def hist(series, title, bins=40, log1p=False):
    s = series.dropna()
    if log1p:
        s = np.log1p(s)
        title = title + " (log1p)"
    plt.figure(figsize=(8,4))
    plt.hist(s, bins=bins)
    plt.title(title)
    plt.xlabel(series.name)
    plt.ylabel("count")
    plt.show()

if "nb_total_de_retraits" in df.columns:
    hist(df["nb_total_de_retraits"], "Distribution du nombre total de retraits")

if "montant_total" in df.columns:
    hist(df["montant_total"], "Distribution du montant total", log1p=True)


In [ ]:
# Scatter montant_total vs nb_total_de_retraits (+ segment B/H si présent)
if "nb_total_de_retraits" in df.columns and "montant_total" in df.columns:
    plt.figure(figsize=(7,6))
    if "dab_hos_site" in df.columns:
        for v in df["dab_hos_site"].dropna().unique():
            d = df[df["dab_hos_site"] == v]
            plt.scatter(d["nb_total_de_retraits"], d["montant_total"], alpha=0.5, label=str(v))
        plt.legend(title="dab_hos_site")
    else:
        plt.scatter(df["nb_total_de_retraits"], df["montant_total"], alpha=0.5)

    plt.xlabel("nb_total_de_retraits")
    plt.ylabel("montant_total")
    plt.title("Montant total vs Nombre total de retraits")
    plt.show()
else:
    print("Colonnes manquantes pour le scatter (nb_total_de_retraits/montant_total).")


## 6) Features non évidentes (les plus utiles pour l’anomalie)

Au lieu de regarder “beaucoup de retraits” ou “montant élevé”, on crée des indicateurs plus intelligents :

### 6.1 Montant moyen par retrait
`montant_moyen_par_retrait = montant_total / nb_total_de_retraits`

- Très bas → retraits fractionnés (évitement de seuils, comportement atypique)
- Très haut → retraits unitaires atypiques

### 6.2 Ratio montant vs attendu
On calcule une **référence globale** (médiane du montant moyen), puis :
`montant_attendu = nb_total_de_retraits * median(montant_moyen_par_retrait)`
`ratio_montant_vs_attendu = montant_total / montant_attendu`

- << 1 : montant “trop faible” vs volume
- >> 1 : montant “trop élevé” vs volume

**Sorties attendues :**
- nouvelles colonnes ajoutées
- stats descriptives + boxplots


In [ ]:
df_feat = df.copy()

if "montant_total" in df_feat.columns and "nb_total_de_retraits" in df_feat.columns:
    df_feat["montant_moyen_par_retrait"] = df_feat["montant_total"] / df_feat["nb_total_de_retraits"].replace(0, np.nan)

    ref = float(np.nanmedian(df_feat["montant_moyen_par_retrait"]))
    df_feat["montant_attendu"] = df_feat["nb_total_de_retraits"] * ref
    df_feat["ratio_montant_vs_attendu"] = df_feat["montant_total"] / df_feat["montant_attendu"].replace(0, np.nan)

    display(df_feat[["montant_moyen_par_retrait","ratio_montant_vs_attendu"]].describe(percentiles=[.01,.05,.5,.95,.99]).T)
else:
    print("Impossible de créer les features (montant_total ou nb_total_de_retraits manquant).")


In [ ]:
# Boxplots simples (matplotlib) pour visualiser les valeurs extrêmes
def boxplot(series, title):
    s = series.dropna()
    plt.figure(figsize=(8,3))
    plt.boxplot(s, vert=False, showfliers=True)
    plt.title(title)
    plt.xlabel(series.name)
    plt.show()

if "montant_moyen_par_retrait" in df_feat.columns:
    boxplot(df_feat["montant_moyen_par_retrait"], "Boxplot — Montant moyen par retrait")

if "ratio_montant_vs_attendu" in df_feat.columns:
    boxplot(df_feat["ratio_montant_vs_attendu"], "Boxplot — Ratio montant vs attendu")


## 7) Détection d’anomalies (Isolation Forest)

**Pourquoi Isolation Forest :**
- Méthode **non supervisée** (pas besoin de labels)
- Détecte des observations rares dans un espace multi-variables (montant, volume, ratios, contexte)

**Ce que fait le code :**
1. Sélectionne les colonnes utiles (numériques + features + catégories)
2. Encode les catégories (LabelEncoder)
3. Remplit les NA (médiane pour numériques)
4. Standardise (StandardScaler)
5. Entraîne Isolation Forest
6. Ajoute :
   - `prediction_anomalie` (1=anomalie, 0=normal)
   - `anomaly_score` (plus grand = plus anormal)

**Sortie attendue :**
- répartition (combien d’anomalies)
- top 30 anomalies triées par score
- scatter plot des anomalies


In [ ]:
model_df = df_feat.copy()

cat_cols = [c for c in ["lib_site_implementation","type_carte","dab_hos_site","typ_gab"] if c in model_df.columns]
num_cols = [c for c in [
    "montant_total","nb_total_de_retraits","type_gab_e_i","code_postale_emplacement","code_banque",
    "montant_moyen_par_retrait","ratio_montant_vs_attendu"
] if c in model_df.columns]

use_cols = cat_cols + num_cols
X_raw = model_df[use_cols].copy()

# Encodage des catégories
encoders = {}
for c in cat_cols:
    le = LabelEncoder()
    X_raw[c] = X_raw[c].astype(str).fillna("NA")
    X_raw[c] = le.fit_transform(X_raw[c])
    encoders[c] = le

# Remplissage NA pour les numériques
for c in num_cols:
    X_raw[c] = pd.to_numeric(X_raw[c], errors="coerce")
    X_raw[c] = X_raw[c].fillna(X_raw[c].median())

# Standardisation
scaler = StandardScaler()
X = scaler.fit_transform(X_raw)

# Modèle (contamination = proportion attendue d'anomalies ; ajuste selon ton contexte)
iso = IsolationForest(
    n_estimators=400,
    contamination=0.08,
    random_state=RANDOM_STATE
)

pred = iso.fit_predict(X)  # 1 normal, -1 anomalie
model_df["prediction_anomalie"] = np.where(pred == -1, 1, 0)

# Score d'anomalie (plus grand = plus anormal)
model_df["anomaly_score"] = -iso.score_samples(X)

model_df["prediction_anomalie"].value_counts()


In [ ]:
# Visualisation : anomalies dans l'espace montant_total / nb_total_de_retraits
if "nb_total_de_retraits" in model_df.columns and "montant_total" in model_df.columns:
    plt.figure(figsize=(7,6))
    normal = model_df[model_df["prediction_anomalie"] == 0]
    anom = model_df[model_df["prediction_anomalie"] == 1]

    plt.scatter(normal["nb_total_de_retraits"], normal["montant_total"], alpha=0.35, label="normal")
    plt.scatter(anom["nb_total_de_retraits"], anom["montant_total"], alpha=0.85, label="anomalie")

    plt.xlabel("nb_total_de_retraits")
    plt.ylabel("montant_total")
    plt.title("Anomalies détectées (Isolation Forest)")
    plt.legend()
    plt.show()
else:
    print("Colonnes manquantes pour le scatter anomalies.")


## 8) Donner des raisons simples (interprétabilité)

Un modèle d’anomalie doit être **actionnable**.  
On génère donc une colonne `raison` basée sur des percentiles (faible/élevé) des features clés.

**Ce que fait le code :**
- Calcule des seuils (5% / 95%) sur `montant_moyen_par_retrait` et `ratio_montant_vs_attendu`
- Attribue une raison lisible par ligne

**Sortie attendue :**
- un tableau des anomalies avec `raison`


In [ ]:
# Seuils (percentiles) pour générer des explications simples
q = {}

for col in ["montant_moyen_par_retrait", "ratio_montant_vs_attendu"]:
    if col in model_df.columns:
        q[(col, "p05")] = model_df[col].quantile(0.05)
        q[(col, "p95")] = model_df[col].quantile(0.95)

def reason(row):
    reasons = []
    if "montant_moyen_par_retrait" in row and pd.notna(row["montant_moyen_par_retrait"]):
        if ("montant_moyen_par_retrait","p05") in q and row["montant_moyen_par_retrait"] < q[("montant_moyen_par_retrait","p05")]:
            reasons.append("montant moyen très faible (fractionnement possible)")
        if ("montant_moyen_par_retrait","p95") in q and row["montant_moyen_par_retrait"] > q[("montant_moyen_par_retrait","p95")]:
            reasons.append("montant moyen très élevé (retraits unitaires atypiques)")

    if "ratio_montant_vs_attendu" in row and pd.notna(row["ratio_montant_vs_attendu"]):
        if ("ratio_montant_vs_attendu","p05") in q and row["ratio_montant_vs_attendu"] < q[("ratio_montant_vs_attendu","p05")]:
            reasons.append("montant total faible vs attendu (incohérence volume/montant)")
        if ("ratio_montant_vs_attendu","p95") in q and row["ratio_montant_vs_attendu"] > q[("ratio_montant_vs_attendu","p95")]:
            reasons.append("montant total élevé vs attendu (incohérence volume/montant)")

    if "dab_hos_site" in row and str(row["dab_hos_site"]) == "H":
        reasons.append("hors site (profil d'exposition différent)")

    return " | ".join(reasons) if reasons else "profil rare multi-variables"

# Appliquer seulement sur les anomalies pour être plus lisible
anoms = model_df[model_df["prediction_anomalie"] == 1].copy()
anoms["raison"] = anoms.apply(reason, axis=1)

cols_show = [c for c in [
    "num_automate","lib_site_implementation","code_banque","type_carte",
    "montant_total","nb_total_de_retraits","montant_moyen_par_retrait","ratio_montant_vs_attendu",
    "dab_hos_site","type_gab_e_i","code_postale_emplacement",
    "anomaly_score","raison"
] if c in anoms.columns]

anoms.sort_values("anomaly_score", ascending=False)[cols_show].head(30)


## 9) Validation non supervisée (sans matrice de confusion)

Sans label, on vérifie la **cohérence** du résultat en comparant :
- la distribution des variables pour les anomalies vs le reste

**Sortie attendue :**
- tableau comparatif (médianes, p95)
- conclusion qualitative : les anomalies sont “différentes” sur des indicateurs clés


In [ ]:
anoms = model_df[model_df["prediction_anomalie"] == 1]
normal = model_df[model_df["prediction_anomalie"] == 0]

cols_check = [c for c in [
    "montant_total","nb_total_de_retraits","montant_moyen_par_retrait","ratio_montant_vs_attendu"
] if c in model_df.columns]

summary = pd.DataFrame({
    "median_normal": normal[cols_check].median(),
    "median_anom": anoms[cols_check].median(),
    "p95_normal": normal[cols_check].quantile(0.95),
    "p95_anom": anoms[cols_check].quantile(0.95),
})
summary


## 10) Stabilité (option pro)

On refait Isolation Forest avec plusieurs valeurs de `contamination` (ex : 5%, 8%, 12%) et on regarde si les **top anomalies** restent les mêmes.

**Pourquoi :**
- Les anomalies “sérieuses” sont souvent détectées même si on change légèrement le paramètre.

**Sortie attendue :**
- taille de l’intersection des top-30 entre paramètres


In [ ]:
def top_ids_for_contamination(cont, top_k=30):
    iso_tmp = IsolationForest(n_estimators=400, contamination=cont, random_state=RANDOM_STATE)
    pred_tmp = iso_tmp.fit_predict(X)
    score_tmp = -iso_tmp.score_samples(X)
    tmp = model_df.copy()
    tmp["score_tmp"] = score_tmp
    if "num_automate" in tmp.columns:
        return set(tmp.sort_values("score_tmp", ascending=False)["num_automate"].head(top_k))
    else:
        # fallback si num_automate absent
        return set(tmp.sort_values("score_tmp", ascending=False).head(top_k).index)

tops_05 = top_ids_for_contamination(0.05)
tops_08 = top_ids_for_contamination(0.08)
tops_12 = top_ids_for_contamination(0.12)

{
    "top_commun_05_08": len(tops_05 & tops_08),
    "top_commun_08_12": len(tops_08 & tops_12),
    "top_commun_05_12": len(tops_05 & tops_12),
}


## 11) Export vers un dataset Dataiku (optionnel)

**Ce que fait le code :**
- Écrit le top anomalies vers un dataset Dataiku de sortie.

✅ **Action pour toi :**
- Crée un dataset vide de sortie dans Dataiku (schema auto) nommé `DAB_ANOMALIES_OUT` (ou ton nom).
- Modifie `OUT_DATASET_NAME`.

**Sortie attendue :**
- un dataset Dataiku rempli avec le top anomalies


In [ ]:
# 🔧 Optionnel : écrire vers Dataiku
OUT_DATASET_NAME = "DAB_ANOMALIES_OUT"

top_out = anoms.sort_values("anomaly_score", ascending=False)[cols_show].head(200).copy()

try:
    out_ds = dataiku.Dataset(OUT_DATASET_NAME)
    out_ds.write_with_schema(top_out)
    print("✅ Écrit dans Dataiku :", OUT_DATASET_NAME, "| lignes =", len(top_out))
except Exception as e:
    print("⚠️ Export Dataiku non effectué. Raison :", e)
    print("👉 Si tu veux exporter : crée le dataset de sortie dans Dataiku et vérifie son nom.")
